<a href="https://colab.research.google.com/github/Catisyf/Python-projects-chez-Toucan/blob/main/back_fill_text_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -U sentence-transformers #sentence-transformers model

In [ ]:
pip install Faker #create fake data 

In [ ]:
!ls '/content/drive/My Drive/crm_contacts.csv'

'/content/drive/My Drive/crm_contacts.csv'


In [3]:
import pandas as pd
import numpy as np
import re
from scipy import spatial
from collections import Counter 
from faker import Faker

In [4]:
df_contacts = pd.read_csv('/content/drive/My Drive/crm_contacts.csv', na_values=' ') #replace blank cells with na
df_persona = pd.read_csv('/content/drive/My Drive/buyer_persona_definition.csv')

## **Data Preparation**

In [5]:
df_contacts = df_contacts.replace('?', 'np.Nan').dropna() #remove null

In [ ]:
df_contacts = df_contacts[~df_contacts.jobtitle.str.contains('(site)|(try-for-free)')] #remove rows with irrelevant info

In [7]:
 stopwords = ['of', 'de', 'du', 'for', 'For', 'the', 'et', "d'"] #create stopwords list
space = ' '
stopwords_dict = Counter(stopwords)

In [8]:
#function to remove stopwords from job title
def remove_stopwords (df_name, column_name):
    clean = [space.join([word for word in name.split() if word not in stopwords_dict]) \
             for name in df_name[column_name]]
    return clean

In [9]:
#function to remove special characters from job title
def remove_sign(df_name, column_name):
  clean = [re.sub('[|!@#$-.&/_+={}()]', ' ', text) for text in df_name[column_name]]
  return clean

In [10]:
#add cleaned job title to datasets
df_persona['clean_title'] = remove_stopwords(df_persona, 'job_title')
df_contacts['clean_title'] = remove_stopwords(df_contacts, 'jobtitle')

In [11]:
df_persona['clean_title'] = remove_sign(df_persona, 'clean_title')
df_contacts['clean_title'] = remove_sign(df_contacts, 'clean_title')

## **NLP model for matching**

In [ ]:
#pre-trained sentence transformer model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [13]:
def text_vectorizer (df_name, column_name):
  vector = [model.encode(title) for title in df_name[column_name]]
  
  return vector

In [14]:
#add vector to datasets
df_persona['vector'] = text_vectorizer (df_persona, 'clean_title')
df_contacts['vector'] = text_vectorizer (df_contacts, 'clean_title')

In [17]:
#reset index 
df_contacts = df_contacts.reset_index(drop = True)

In [19]:
#function to calculate cosine similarity
def cosine_similarity(v1,v2):
  return 1-spatial.distance.cosine(v1, v2)

In [20]:
#find a match for each job title in CRM data using the max value of cosine similarity: 

similarity = {}
i = 0
for vec1 in df_contacts['vector']:
  similarity_persona = []
  for vec2 in df_persona['vector']:
    similarity_persona.append(cosine_similarity(vec1, vec2))
  
  max_similiarity = max(similarity_persona)
  persona_id = similarity_persona.index(max_similiarity) #
  similarity[i] = [max_similiarity, persona_id]
  i += 1


In [36]:
table = pd.DataFrame.from_dict(similarity, orient='index')
table.columns =['similarity_score', 'persona_job_index']

In [37]:
#join tables using index 
table_full = table.join(df_contacts[['jobtitle', 'email', 'vector']]\
                       ).merge(df_persona[['buyer_persona', 'job_title', 'vector']], \
                               left_on = 'persona_job_index', right_index = True)

In [38]:
#rename table column
table_full = table_full.rename(columns={'jobtitle': "job_title_CRM", 'job_title': 'job_title_toucan'})

In [49]:
table_full = table_full[['job_title_CRM', 'email', 'job_title_toucan', 'buyer_persona', 'similarity_score']]

#filter with high similarity score to get good matches 
table_good_score = table_full.loc[table_full['similarity_score'] > 0.85]

#query filtered data
table_filtered_results = table_full.query('job_title_CRM not in @table_good_score.job_title_CRM')

In [61]:
#create fake emails to anonymise confidential data
faker = Faker()
fake_email = [faker.company_email() for i in range(349)]
table_good_score['email'] = fake_email

In [63]:
table_good_score

,job_title_CRM,email,job_title_toucan,buyer_persona,similarity_score
4146,Founders,james30@kelly-smith.com,Founder,Business Leader,0.902885
24,Manager of Sales,thomaswebb@diaz.org,Sales Director,Business Leader,0.932112
812,Sales Executive,james88@perez-good.com,Sales Director,Business Leader,0.917183
854,Vendor Manager,brownlaura@erickson.com,Sales Director,Business Leader,0.920250
1907,Sales Manager,ashley68@duarte.com,Sales Director,Business Leader,0.936844
...,...,...,...,...,...
3241,Consultor de Business Intelligence,gibsonerika@nicholson-price.org,Consultant Business Intelligence,Practitioner,0.986818
5007,Expert Business Intelligence (Maltem Consulting),rsmith@wu.info,Consultant Business Intelligence,Practitioner,0.869026
12371,Consultante Business Intelligence,hayessteve@buck-rodriguez.com,Consultant Business Intelligence,Practitioner,0.999221
16493,Consultante Pilotage & Business Intelligencea,edwardshaw@freeman.info,Consultant Business Intelligence,Practitioner,0.896643
